## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,61.34,79,1,1.01,clear sky
1,1,Qaanaaq,GL,77.4840,-69.3632,19.06,59,63,8.12,broken clouds
2,2,Hilo,US,19.7297,-155.0900,87.28,84,90,9.22,moderate rain
3,3,Krasnoselkup,RU,65.7000,82.4667,33.31,90,66,8.16,broken clouds
4,4,Ikongo,MG,-21.8833,47.4333,60.04,96,27,2.59,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,61.34,79,1,1.01,clear sky
4,4,Ikongo,MG,-21.8833,47.4333,60.04,96,27,2.59,scattered clouds
5,5,Touros,BR,-5.1989,-35.4608,76.10,82,39,15.70,scattered clouds
6,6,Corsicana,US,32.0954,-96.4689,80.96,48,1,5.75,clear sky
8,8,Georgetown,MY,5.4112,100.3354,80.89,87,20,3.44,few clouds
9,9,Rikitea,PF,-23.1203,-134.9692,73.78,72,2,19.42,clear sky
10,10,Port Alfred,ZA,-33.5906,26.8910,64.33,91,25,19.57,scattered clouds
11,11,Nador,MA,35.1740,-2.9287,64.65,88,0,0.00,clear sky
12,12,Kapaa,US,22.0752,-159.3190,84.18,73,75,5.01,light rain
13,13,Palmer,US,42.1584,-72.3287,57.85,79,1,0.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                8
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,61.34,79,1,1.01,clear sky
4,4,Ikongo,MG,-21.8833,47.4333,60.04,96,27,2.59,scattered clouds
5,5,Touros,BR,-5.1989,-35.4608,76.10,82,39,15.70,scattered clouds
6,6,Corsicana,US,32.0954,-96.4689,80.96,48,1,5.75,clear sky
8,8,Georgetown,MY,5.4112,100.3354,80.89,87,20,3.44,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Jamestown,US,61.34,42.0970,-79.2353,clear sky,
4,Ikongo,MG,60.04,-21.8833,47.4333,scattered clouds,
5,Touros,BR,76.10,-5.1989,-35.4608,scattered clouds,
6,Corsicana,US,80.96,32.0954,-96.4689,clear sky,
8,Georgetown,MY,80.89,5.4112,100.3354,few clouds,
9,Rikitea,PF,73.78,-23.1203,-134.9692,clear sky,
10,Port Alfred,ZA,64.33,-33.5906,26.8910,scattered clouds,
11,Nador,MA,64.65,35.1740,-2.9287,clear sky,
12,Kapaa,US,84.18,22.0752,-159.3190,light rain,
13,Palmer,US,57.85,42.1584,-72.3287,clear sky,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace = True)
# clean_hotel_df = hotel_df.dropna(subset = ["Hotel Name"], inplace = True)

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp,dissipating = False,
             max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))